In [5]:
import onnxruntime
import numpy as np

In [4]:
# Load the ONNX model
model_wo_params = onnxruntime.InferenceSession("/home/nvidia/dev/img_new/resnet_v1.onnx", 
                                               providers=['CPUExecutionProvider', 'AzureExecutionProvider'])
model_with_params = onnxruntime.InferenceSession("/home/nvidia/dev/img_new/resnet_v1_with_params.onnx",
                                                 providers=['CPUExecutionProvider', 'AzureExecutionProvider'])

In [8]:
img = np.ones((1,3,160,160),dtype=np.float32)


In [18]:
# Run inference
output_wo_params = np.array(model_wo_params.run(None, {"input": img})[0][0])
output_with_params = np.array(model_with_params.run(None, {"input": img})[0][0])

In [21]:
np.linalg.norm(output_wo_params)

1.0

In [22]:
out_orig_pt = np.load("/home/nvidia/dev/img_new/scripts/ones_orig.npy")
out_orig_pt.shape

(1, 512)

In [24]:
out_pt = out_orig_pt[0]
np.linalg.norm(output_with_params - out_pt)

3.0681604e-06

In [31]:
import onnx
import onnx.checker


# Load the ONNX model
model = onnx.load("/home/nvidia/dev/img_new/resnet_v1_with_params.onnx")

# Check the model for errors
checker = onnx.checker.check_model(model)

# Print the model's input and output nodes
print(checker.get_input_names())
print(checker.get_output_names())

AttributeError: 'NoneType' object has no attribute 'get_input_names'

In [32]:
import onnx
import onnx.helper

# Load the ONNX model
model = onnx.load("/home/nvidia/dev/img_new/resnet_v1_with_params.onnx")

# Print the model's graph
print(onnx.helper.print_model(model))

AttributeError: module 'onnx.helper' has no attribute 'print_model'

In [2]:
import onnx
"""
onnx.onnx_ml_pb2.ModelProto is a Python class that represents an 
ONNX model in Protocol Buffer (protobuf) format.

When you load an ONNX model using the onnx.load() function, it returns an 
instance of the ModelProto class, which contains the model's graph, 
inputs, outputs, and other properties.
"""
model = onnx.load("/home/nvidia/dev/img_new/resnet_v1_with_params.onnx")
type(model)

onnx.onnx_ml_pb2.ModelProto

In [3]:
# Get the input and output nodes
input_nodes = model.graph.input
output_nodes = model.graph.output

# Print the input and output nodes
for node in input_nodes:
    print(f"Input Node: {node.name}")
    print(f"  Type: {node.type}")
    print(f"  Shape: {node.type.tensor_type.shape.dim}")

for node in output_nodes:
    print(f"Output Node: {node.name}")
    print(f"  Type: {node.type}")
    print(f"  Shape: {node.type.tensor_type.shape.dim}")

Input Node: input
  Type: tensor_type {
  elem_type: 1
  shape {
    dim {
      dim_param: "input_dynamic_axes_1"
    }
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 160
    }
    dim {
      dim_value: 160
    }
  }
}

  Shape: [dim_param: "input_dynamic_axes_1"
, dim_value: 3
, dim_value: 160
, dim_value: 160
]
Output Node: embedding
  Type: tensor_type {
  elem_type: 1
  shape {
    dim {
      dim_param: "embedding_dynamic_axes_1"
    }
    dim {
      dim_param: "Divembedding_dim_1"
    }
  }
}

  Shape: [dim_param: "embedding_dynamic_axes_1"
, dim_param: "Divembedding_dim_1"
]
